In [ ]:
import pandas as pd
import scanpy as sc
import plotnine as p9
import liana as li
import os
import anndata as ad
import numpy as np
from scipy.stats import combine_pvalues
from matplotlib import pyplot as plt
import seaborn as sns

from mudata import MuData
from itertools import product

In [ ]:
root_path = os.getcwd()
results_folder = os.path.join(root_path, 'results/')

In [ ]:
def select_slide(adata, s, batch_key="sample"):
    r"""This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param batch_key: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[batch_key].isin([s]), :].copy()
    s_keys = list(slide.uns["spatial"].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns["spatial"] = {s_spatial: slide.uns["spatial"][s_spatial]}

    return slide

In [ ]:
results_folder

In [ ]:
### Read the deconvolution analysis output adata -> celltype5 level for highest resolution
adata_file_location = 'deconvolution_bbknn/celltype5_updated_v1/cell2location_map/sp.h5ad'
adata = ad.read(os.path.join(results_folder, adata_file_location))

In [ ]:
sample_names = adata.obs['readout_id'].unique().tolist()

In [ ]:
sample_names

In [ ]:
mdata_list = []

for current_sample in sample_names: 
    
    current_adata = select_slide(adata, s = current_sample, batch_key='readout_id')
    df_cellabundance = current_adata.obsm['q05_cell_abundance_w_sf'].copy()
    df_cellabundance.columns =  adata.uns['mod']['factor_names']
    
    plot, _ = li.ut.query_bandwidth(coordinates=current_adata.obsm['spatial'], start=0, end=500, interval_n=20)
    plot + p9.scale_y_continuous(breaks=range(0, 500, 5))
    
    li.ut.spatial_neighbors(current_adata, bandwidth=100, cutoff=0.1, kernel='gaussian', set_diag=True)
    li.pl.connectivity(current_adata, idx=0, size=1.3, figure_size=(6, 5))
    
    
    comps = li.ut.obsm_to_adata(current_adata, obsm_key = 'q05_cell_abundance_w_sf', df = df_cellabundance)
    
    mdata = MuData({"comps_1":comps, "comps_2":comps})
    mdata.obsp = current_adata.obsp
    mdata.uns = current_adata.uns
    mdata.obsm = current_adata.obsm
    
    interactions = list(product(comps.var.index, comps.var.index))
    
    li.mt.bivar(mdata,
            x_mod="comps_1",
            y_mod="comps_2",
            x_transform=sc.pp.scale,
            y_transform=sc.pp.scale,
            function_name="cosine",
            interactions=interactions,
            mask_negatives=True,
            add_categories=True,
            x_use_raw=False,
            y_use_raw=False,
            xy_sep="<->",
            )
    
    mdata_list.append(mdata)

In [ ]:
mdata_list_untreated = [mdata for mdata in mdata_list if 'Untreated' in mdata.obs['comps_1:CONDITION'].values] 
mdata_list_FAP_LTBR =  [mdata for mdata in mdata_list if 'FAP_LTBR' in mdata.obs['comps_1:CONDITION'].values] 

In [ ]:
treat_id=['V43J24-078_A1_B06',
 'V43A13-374_A1_B19',
 'V43J11-302_A1_B08',
 'V43J19-050_A1_B07',
 'V42D20-002_A1_B08',
 'V43A11-284_A1_B16',
 'V42D20-025_A1_B16',
 'V43J19-319_A1_B09']

In [ ]:
treat_idk=['V43J24-078_A1_B06', 'V43J19-050_A1_B07',
            'V43J11-302_A1_B08',
            'V43A11-284_A1_B16',
 'V43A13-374_A1_B19']

In [ ]:
concatenated_df = pd.concat([df.mod['local_scores'].var[['mean']].set_index(df.mod['local_scores'].var[['mean']].index) for df in mdata_list_FAP_LTBR], axis=1, keys=range(len(mdata_list_FAP_LTBR)))

In [ ]:
concatenated_df.columns=treat_id

In [ ]:
average_means = pd.DataFrame(concatenated_df.loc[:,treat_idk].median(axis=1))

In [ ]:
# Split the index into two parts using the str.split method
split_index = pd.Series(average_means.index).str.split('<->', expand=True)
average_means['cellTypeA'] = list(split_index[0])
average_means['cellTypeB'] = list(split_index[1])

In [ ]:
average_means.columns=['Interact','cellTypeA','cellTypeB']

In [ ]:
root_path = os.getcwd()
results_folder = os.path.join(root_path, 'figures/')

In [ ]:
results_folder

In [ ]:
average_means.to_csv(results_folder+'ALL_Interactions_Celltypes_avgmean.tsv',sep='\t')

In [ ]:
average_means.loc[average_means['cellTypeA']=='endothelial cell of high endothelial venule',:].sort_values(by='Interact', ascending=False)

In [ ]:
average_means.loc[average_means['cellTypeA']=='B cell',:].sort_values(by='Interact', ascending=False)

In [ ]:
average_means.loc[average_means['cellTypeA']=='CD8-positive, alpha-beta resource T cell',:].sort_values(by='Interact', ascending=False)

In [ ]:
df=average_means.loc[average_means['cellTypeA']=='CD8-positive, alpha-beta resource T cell',:].sort_values(by='Interact', ascending=False)#.plot.bar()
df.set_index("cellTypeB", inplace=True)


In [ ]:
ax = df.plot.bar(rot=0, color='grey', figsize=(6, 4))
plt.title("CD8+ resource T cell", fontsize=10)
plt.xlabel("cellTypeB", fontsize=10)
plt.ylabel("Interaction", fontsize=10)
plt.xticks(rotation=90, ha='center', fontsize=10) 
plt.savefig(results_folder+'ALL_Interactions_Celltypes_avgmean_CD8resourceTcell.pdf', format="pdf")
plt.show()

In [ ]:
average_means.loc[average_means['cellTypeA']=='fibroblast',:].sort_values(by='Interact', ascending=False).plot.bar()

In [ ]:
df=average_means.loc[average_means['cellTypeA']=='endothelial cell of high endothelial venule',:].sort_values(by='Interact', ascending=False).copy()
df.set_index("cellTypeB", inplace=True)


ax = df.plot.bar(rot=0, color='grey', figsize=(6, 4))
plt.title("HEV", fontsize=10)
plt.xlabel("cellTypeB", fontsize=10)
plt.ylabel("Interaction", fontsize=10)
plt.xticks(rotation=90, ha='center', fontsize=10) 
plt.savefig(results_folder+'ALL_Interactions_Celltypes_avgmean_HEV.pdf', format="pdf")
plt.show()

In [ ]:
i=0


In [ ]:
for i in range(len(mdata_list_FAP_LTBR)):
    print (mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0])
    sc.pl.spatial(mdata_list_FAP_LTBR[i].mod['local_scores'], 
                  color=['endothelial cell of high endothelial venule<->B cell', 
                         'endothelial cell of high endothelial venule<->naive thymus-derived CD8-positive, alpha-beta T cell', 
                         'endothelial cell of high endothelial venule<->CD8-positive, alpha-beta resource T cell',
                        'endothelial cell of high endothelial venule<->CCR7-positive myeloid dendritic cell', ], 
                  size=1.4, cmap="coolwarm", vmax=1, vmin=-1, ncols=2, save='HEV-interactions-'+mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0]+'.pdf')

In [ ]:
for i in range(len(mdata_list_FAP_LTBR)):
    print (mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0])
    sc.pl.spatial(mdata_list_FAP_LTBR[i].mod['local_scores'], color=['fibroblast<->endothelial cell of high endothelial venule', 'fibroblast<->CD8-positive, alpha-beta resource T cell'], size=1.4, cmap="coolwarm", vmax=1, vmin=-1)

In [ ]:
for i in range(len(mdata_list_FAP_LTBR)):
    print (mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0])
    sc.pl.spatial(mdata_list_FAP_LTBR[i].mod['local_scores'], layer='cats', color=['endothelial cell of high endothelial venule<->B cell', 'endothelial cell of high endothelial venule<->CD8-positive, alpha-beta resource T cell'], cmap='coolwarm')

In [ ]:
mdata.mod['local_scores'].var.sort_values("mean", ascending=False).head(50)

In [ ]:
pivot_df=average_means.pivot(columns = 'cellTypeA', index = "cellTypeB", values="Interact")

In [ ]:
#pivot_df = mdata.mod['local_scores'].var.pivot(columns = 'x', index = "y", values="mean")

In [ ]:
pivot_df

In [ ]:
pivot_df.fillna(0, inplace=True)
# Create the clustermap
cg = sns.clustermap(pivot_df, cmap="vlag", figsize=(14, 14), linewidths=.75, 
                    dendrogram_ratio=(.125, .01), vmin=-1, vmax=1, center=0, annot=True,annot_kws={"size": 6})
cg.ax_row_dendrogram.set_visible(False) #suppress row dendrogram
cg.ax_col_dendrogram.set_visible(False) #suppress row dendrogram

### BBKNN Celltype 2

In [ ]:
root_path = os.getcwd()
results_folder = os.path.join(root_path, 'results/')

In [ ]:
adata_file_location = 'deconvolution_bbknn/celltype2_updated_v1/cell2location_map/sp.h5ad'
adata = ad.read(os.path.join(results_folder, adata_file_location))
sample_names = adata.obs['readout_id'].unique().tolist()


In [ ]:

mdata_list = []

for current_sample in sample_names: 
    
    current_adata = select_slide(adata, s = current_sample, batch_key='readout_id')
    df_cellabundance = current_adata.obsm['q05_cell_abundance_w_sf'].copy()
    df_cellabundance.columns =  adata.uns['mod']['factor_names']
    
    plot, _ = li.ut.query_bandwidth(coordinates=current_adata.obsm['spatial'], start=0, end=500, interval_n=20)
    plot + p9.scale_y_continuous(breaks=range(0, 500, 5))
    
    li.ut.spatial_neighbors(current_adata, bandwidth=100, cutoff=0.1, kernel='gaussian', set_diag=True)
    li.pl.connectivity(current_adata, idx=0, size=1.3, figure_size=(6, 5))
    
    
    comps = li.ut.obsm_to_adata(current_adata, obsm_key = 'q05_cell_abundance_w_sf', df = df_cellabundance)
    
    mdata = MuData({"comps_1":comps, "comps_2":comps})
    mdata.obsp = current_adata.obsp
    mdata.uns = current_adata.uns
    mdata.obsm = current_adata.obsm
    
    interactions = list(product(comps.var.index, comps.var.index))
    
    li.mt.bivar(mdata,
            x_mod="comps_1",
            y_mod="comps_2",
            x_transform=sc.pp.scale,
            y_transform=sc.pp.scale,
            function_name="cosine",
            interactions=interactions,
            mask_negatives=True,
            add_categories=True,
            x_use_raw=False,
            y_use_raw=False,
            xy_sep="<->",
            )
    
    mdata_list.append(mdata)

In [ ]:
mdata_list_untreated = [mdata for mdata in mdata_list if 'Untreated' in mdata.obs['comps_1:CONDITION'].values] 
mdata_list_FAP_LTBR =  [mdata for mdata in mdata_list if 'FAP_LTBR' in mdata.obs['comps_1:CONDITION'].values] 
treat_id=['V43J24-078_A1_B06',
 'V43A13-374_A1_B19',
 'V43J11-302_A1_B08',
 'V43J19-050_A1_B07',
 'V42D20-002_A1_B08',
 'V43A11-284_A1_B16',
 'V42D20-025_A1_B16',
 'V43J19-319_A1_B09']
treat_idk=['V43J24-078_A1_B06', 'V43J19-050_A1_B07',
            'V43J11-302_A1_B08',
            'V43A11-284_A1_B16',
 'V43A13-374_A1_B19']
concatenated_df = pd.concat([df.mod['local_scores'].var[['mean']].set_index(df.mod['local_scores'].var[['mean']].index) for df in mdata_list_FAP_LTBR], axis=1, keys=range(len(mdata_list_FAP_LTBR)))
concatenated_df.columns=treat_id

average_means = pd.DataFrame(concatenated_df.loc[:,treat_idk].median(axis=1))
# Split the index into two parts using the str.split method
split_index = pd.Series(average_means.index).str.split('<->', expand=True)

# Assign the split parts to new columns in the DataFrame
average_means['cellTypeA'] = list(split_index[0])
average_means['cellTypeB'] = list(split_index[1])
average_means.columns=['Interact','cellTypeA','cellTypeB']

root_path = os.getcwd()


In [ ]:
results_folder = os.path.join(root_path, 'figures/')
average_means.to_csv(results_folder+'ALL_Interactions_Celltypes-lev2_avgmean.tsv',sep='\t')


In [ ]:
average_means.loc[average_means['cellTypeA']=='endothelial cell of high endothelial venule',:].sort_values(by='Interact', ascending=False)


In [ ]:
average_means.loc[average_means['cellTypeA']=='B cell',:].sort_values(by='Interact', ascending=False)


In [ ]:
df=average_means.loc[average_means['cellTypeA']=='CD8-positive, alpha-beta T cell',:].sort_values(by='Interact', ascending=False)
df.set_index("cellTypeB", inplace=True)

ax = df.plot.bar(rot=0, color='grey', figsize=(4, 3))
plt.title("CD8+ T cell", fontsize=10)
plt.xlabel("cellTypeB", fontsize=10)
plt.ylabel("Interaction", fontsize=10)
plt.xticks(rotation=90, ha='center', fontsize=10)  # ha='center' aligns the labels horizontally
plt.savefig(results_folder+'ALL_Interactions_Celltypes_avgmean_CD8Tcell.pdf', format="pdf")


In [ ]:
df=average_means.loc[average_means['cellTypeA']=='endothelial cell of high endothelial venule',:].sort_values(by='Interact', ascending=False).copy()
df.set_index("cellTypeB", inplace=True)

ax = df.plot.bar(rot=0, color='grey', figsize=(5, 3))
plt.title("HEV", fontsize=10)
plt.xlabel("cellTypeB", fontsize=10)
plt.ylabel("Interaction", fontsize=10)
plt.xticks(rotation=90, ha='center', fontsize=10)  # ha='center' aligns the labels horizontally
plt.savefig(results_folder+'ALL_Interactions_Celltypes-lev2_avgmean_HEV.pdf', format="pdf")
plt.show()

i=0



In [ ]:
for i in range(len(mdata_list_FAP_LTBR)):
    print (mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0])
    sc.pl.spatial(mdata_list_FAP_LTBR[i].mod['local_scores'], 
                  color=['endothelial cell of high endothelial venule<->B cell', 
                         'endothelial cell of high endothelial venule<->CD8-positive, alpha-beta T cell', 
                         'endothelial cell of high endothelial venule<->CD4-positive, alpha-beta T cell',
                        'endothelial cell of high endothelial venule<->myeloid dendritic cell',
                        'endothelial cell of high endothelial venule<->pericyte',
                        'endothelial cell of high endothelial venule<->malignant cell',], 
                  size=1.4, cmap="coolwarm", vmax=1, vmin=-1, ncols=2, save='HEV-interactions-celltype2-'+mdata_list_FAP_LTBR[i].obs['comps_1:readout_id'][0]+'.pdf')

In [ ]:
! jupyter nbconvert --to html 08_y_Deconvolution_C2L_Downstream_Coloc_EnrichAnnotations-updated.ipynb